In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv

load_dotenv("./.env")
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import BaseMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from typing import Annotated, Sequence

class AgentState(TypedDict):
    """State for the agent."""
    messages: Annotated[Sequence[BaseMessage], add_messages]

llm = ChatOpenAI(model="gpt-4o")

@tool
def add(x: int , y:int) -> int:
    """Add two nombers and return the resutl."""
    return x + y

@tool
def subtract(x: int, y: int) -> int:
    """Subtract two numbers and return the result."""
    return x - y

tools =[add,subtract]
llm_with_tools = llm.bind_tools(tools)

sys_msg = "You are a helpful assistant that can add and subtract numbers. You can use the tools 'add' and 'subtract' to perform these operations."

def assistant(state: AgentState) -> AgentState:
    """Assistant function that processes the state and returns the messages."""
    response = llm_with_tools.invoke([sys_msg] + state["messages"])
    return {"messages": response}

def conditional_edge(state: AgentState) -> str:
    """Determine the next node based on the state."""
    return "add" 

toolNode = ToolNode(tools)




In [ ]:

graph = StateGraph(AgentState)
graph.add_node("assistant", assistant)
graph.add_node("toolNode", toolNode)
graph.add_edge(START, "assistant")
graph.add_conditional_edges(
    "assistant",
    conditional_edge {
        "add": "toolNode",
        "subtract":"toolNode",
    }
)